In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('landmarks.csv')

# pretreatment

We shift point 0 (wrist) to (0,0) and normalize so that the length between point 0 and 9 (the distance between the wrist and the joint connecting the middle finger to the palm) is 1.

In [ ]:
l = np.sqrt((df.x_9-df.x_0)**2 + (df.y_9-df.y_0)**2)
for i in range(20,0,-1):
  df['x_'+str(i)] = (df['x_'+str(i)]-df.x_0)/l
  df['y_'+str(i)] = (df['y_'+str(i)]-df.y_0)/l

# ML treatment

In [ ]:
# We define the predictor variables and the target one

predictor=[]
for i in range(21):
  predictor.append("x_"+str(i))
  predictor.append("y_"+str(i))

target = ['lettre']

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
X=df[predictor]
y = df[target]

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# We separate into datasets for the training and for the test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Definition of the model and learning

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

### Test of the model

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

17/17 [==============================] - 0s 2ms/step - loss: 0.0327 - accuracy: 0.9943
Loss: 0.032673243433237076, Accuracy: 0.9942638874053955
